In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir(globals()['_dh'][0])
os.chdir('..')
# print(os.path.abspath(os.curdir))

In [2]:
from src.torchwavenet import Wave
from src.torchdataset import RFMixtureDatasetBase
from omegaconf import OmegaConf
import torch
from torch.utils.data import DataLoader

import hiddenlayer as hl

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [4]:
dataset = RFMixtureDatasetBase(root_dir="npydataset/Dataset_QPSK_CommSignal2_Mixture")
loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [5]:
batch = next(iter(loader))

In [6]:
cfg = OmegaConf.load("src/configs/wavenet.yml")
model = Wave(cfg.model).to(device)

In [7]:
hl.build_graph(model, batch["sample_mix"].to(device))

/home/ogechukwu.kanu/software/miniconda3/envs/rftorch/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1708025824022/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


AttributeError: module 'torch.onnx' has no attribute '_optimize_trace'